In [1]:
# student_id = '2023110214'
student_id = '2023220397'

In [2]:
from competition_utils import *
from training_args import *

model_name = f'submission_{student_id}'
module_path = f"models.{model_name}.{model_name}"
module = importlib.import_module(module_path)
globals()[model_name] = getattr(module, model_name)

Project_Name = 'Competition'
model_dir = 'models'
output_root = 'output'

SAVE_RESULT = True
SAVE_N = 1500
vis_root = f'vis/TEST_OUTPUTS_{datetime.now().strftime("%y%m%d_%H%M%S")}'

Dataset_root = 'Datasets'
Dataset_Name_list = [
                    # 'VOC',
                    'ETIS',
                    'CVPPP',
                    'CFD',
                    'CarDD',
]
number_of_classes_dict = {'VOC':21,
                        'ETIS':2,
                        'CVPPP':2,
                        'CFD':2,
                        'CarDD':2
                        }
epochs = 30
EARLY_STOP = 100
batch_size = 16
EXCLUDE_BACKGROUND = True
BINARY_SEG = None
THRESHOLD = 0.5
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
now = datetime.now()
Experiments_Time=now.strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)

eval_columns=['Experiment Time','Train Time', 'Dataset Name', 'Model Name', 'Val Loss', 'Test Loss', 'IoU', 'Dice',  'Precision', 'Recall', 'Total Params','Train-Predction Time']

df = pd.DataFrame(index=None, columns=eval_columns)
output_root = f'{output_root}/output_{Experiments_Time}'
os.makedirs(output_root, exist_ok = True)

seed = iteration = 1

for j, Dataset_Name in enumerate(Dataset_Name_list):
    print(f'Dataset: {Dataset_Name} ({j+1}/{len(Dataset_Name_list)})')
    control_random_seed(seed)
    Dataset_dir = Dataset_root+'/'+Dataset_Name
    Height, Width = (256, 256)
    in_channels = 3
    number_of_classes = number_of_classes_dict[Dataset_Name]
    BINARY_SEG = True if number_of_classes==2 else False
    exclude_background = EXCLUDE_BACKGROUND
    out_channels = 1 if BINARY_SEG else number_of_classes

    train_image_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/train/Originals/*"))
    train_target_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/train/Masks/*"))
    validation_image_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/val/Originals/*"))
    validation_target_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/val/Masks/*"))
    test_image_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/test/Originals/*"))
    test_target_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/test/Masks/*"))
    print(f'train/val/test: {len(train_image_path_list)}/{len(validation_image_path_list)}/{len(test_image_path_list)}')

    # train_dataset = ImagesDataset(train_image_path_list, train_target_path_list, augment=False)
    train_dataset = ImagesDataset(train_image_path_list, train_target_path_list)
    validation_dataset = ImagesDataset(validation_image_path_list, validation_target_path_list)
    test_dataset = ImagesDataset(test_image_path_list, test_target_path_list)
    test_dataset = ImagesDataset(test_image_path_list, test_target_path_list)
    train_loader = SegDataLoader(
    train_dataset, batch_size=batch_size,
    num_workers=4, pin_memory=True, shuffle=True, drop_last=True, fill_last_batch=False,
    )
    validation_loader = SegDataLoader(
        validation_dataset, batch_size=batch_size, 
        num_workers=4, pin_memory=True,
    )
    test_loader = SegDataLoader(
        test_dataset, batch_size=batch_size, 
        num_workers=4, pin_memory=True,
    )

    print(f'{model_name}', end=' ')
    print(f'Dataset: {Dataset_Name}) ({j+1}/{len(Dataset_Name_list)})', end=' ')
    output_dir = output_root + f'/{model_name}_{Dataset_Name}'
    control_random_seed(seed)
    model=globals()[model_name](in_channels, out_channels)
    model = model.to(device)
    optimizer = Make_Optimizer(model)
    lr_scheduler = Make_LR_Scheduler(optimizer)
    criterion = Make_Loss_Function(number_of_classes)
    if SAVE_RESULT==True:
        os.makedirs(vis_root+f"/{Dataset_Name}", exist_ok=True)
    df = Execute_Experiment(model_name, model, Dataset_Name, train_loader, validation_loader, test_loader, 
                        optimizer, lr_scheduler, criterion, number_of_classes, df, epochs, device, output_dir,
                        BINARY_SEG, exclude_background, out_channels, seed, THRESHOLD, EARLY_STOP, SAVE_RESULT, vis_root+f"/{Dataset_Name}", Experiments_Time
                        )
    
    df.to_csv(output_root+'/'+f'{Project_Name}_'+Experiments_Time+'.csv', index=False, header=True, encoding="cp949")

Experiment Start Time: 250612_104829
Dataset: ETIS (1/4)
train/val/test: 118/39/39
submission_2023220397 Dataset: ETIS) (1/4) Training Start Time: 250612_104830


Training:   0%|          | 0/7 [00:00<?, ?batch/s]

Valiation:   0%|          | 0/3 [00:00<?, ?batch/s]

1EP(250612_104834): T_Loss: 0.687205 V_Loss: 0.642928 IoU: 0.0671 Best Epoch: 1 Loss: 0.642928


Training:   0%|          | 0/7 [00:00<?, ?batch/s]

...

Training:   0%|          | 0/19 [00:00<?, ?batch/s]

Valiation:   0%|          | 0/7 [00:00<?, ?batch/s]

30EP(250612_105320): T_Loss: 0.331813 V_Loss: 0.433815 IoU: 0.3770 
Test Start Time: 250612_105320


Test:   0%|          | 0/7 [00:00<?, ?batch/s]

Best Epoch: 24
Test(250612_105338): Loss: 0.432750 IoU: 0.3917 Dice: 0.5150 Precision: 0.5621 Recall: 0.5751
End 250612_105338


In [ ]:
# Test-08

# Experiment Start Time: 250612_104829
# Dataset: ETIS (1/4)
# train/val/test: 118/39/39
# submission_2023220397 Dataset: ETIS) (1/4) Training Start Time: 250612_104830

# Best Epoch: 25
# Test(250612_104926): Loss: 0.503723 IoU: 0.4022 Dice: 0.4997 Precision: 0.4899 Recall: 0.5673
# End 250612_104926
# Dataset: CVPPP (2/4)
# train/val/test: 486/162/162
# submission_2023220397 Dataset: CVPPP) (2/4) Training Start Time: 250612_104926


# Best Epoch: 30
# Test(250612_105127): Loss: 0.293118 IoU: 0.8469 Dice: 0.9138 Precision: 0.8515 Recall: 0.9940
# End 250612_105127
# Dataset: CFD (3/4)
# train/val/test: 70/24/24
# submission_2023220397 Dataset: CFD) (3/4) Training Start Time: 250612_105127

# Best Epoch: 30
# Test(250612_105211): Loss: 0.574595 IoU: 0.2046 Dice: 0.3368 Precision: 0.2296 Recall: 0.7251
# End 250612_105211
# Dataset: CarDD (4/4)
# train/val/test: 310/104/104
# submission_2023220397 Dataset: CarDD) (4/4) Training Start Time: 250612_105211

# Best Epoch: 24
# Test(250612_105338): Loss: 0.432750 IoU: 0.3917 Dice: 0.5150 Precision: 0.5621 Recall: 0.5751
# End 250612_105338

In [ ]:
# Origin

# Best Epoch: 30
# Test(250607_202805): Loss: 0.657914 IoU: 0.4055 Dice: 0.5255 Precision: 0.4487 Recall: 0.6873
# End 250607_202805
# Dataset: CVPPP (2/4)
# train/val/test: 486/162/162
# submission_2023220397 Dataset: CVPPP) (2/4) Training Start Time: 250607_202806


# Best Epoch: 30
# Test(250607_203007): Loss: 0.402079 IoU: 0.8446 Dice: 0.9124 Precision: 0.8491 Recall: 0.9939
# End 250607_203007
# Dataset: CFD (3/4)
# train/val/test: 70/24/24
# submission_2023220397 Dataset: CFD) (3/4) Training Start Time: 250607_203007


# Best Epoch: 6
# Test(250607_203051): Loss: 0.753741 IoU: 0.0911 Dice: 0.1648 Precision: 0.1001 Recall: 0.6003
# End 250607_203051
# Dataset: CarDD (4/4)
# train/val/test: 310/104/104
# submission_2023220397 Dataset: CarDD) (4/4) Training Start Time: 250607_203051


# Best Epoch: 25
# Test(250607_203220): Loss: 0.573640 IoU: 0.3521 Dice: 0.4765 Precision: 0.5504 Recall: 0.5357
# End 250607_203220